In [ ]:
"""
Implementation of an attention-based model for item recommendation.

Wang, Shoujin, Liang Hu, Longbing Cao, Xiaoshui Huang, Defu Lian, and Wei Liu.
"Attention-based transactional context embedding for next-item recommendation."
In Proceedings of the AAAI conference on artificial intelligence, vol. 32, no. 1. 2018.
"""

In [ ]:
import json
from pathlib import Path
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
import sys

sys.path.append("./../../")
print(os.getcwd())

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tqdm

from choice_learn.basket_models import TripDataset
from choice_learn.basket_models.attn_model import AttentionBasedContextEmbedding
from choice_learn.basket_models.synthetic_dataset import SyntheticDataGenerator

In [ ]:
# Parameters

n_baskets = 1000
epochs = 400
lr = 0.02
embedding_dim = 6
n_negative_samples = 3
assortments_matrix = np.array([[1,0,1,1,0,1,1,0]])
n_items = assortments_matrix.shape[1]


In [ ]:
# Generate synthetic dataset

data_gen = SyntheticDataGenerator(
    proba_complementary_items=0.7,
    proba_neutral_items=0.3,
    noise_proba=0.15,
    items_nest = {0:[0, 1, 2],
                   1: [3, 4, 5],
                   2: [6],
                   3: [7]},
    nests_interactions = [["", "compl", "neutral", "neutral"],
                          ["compl", "", "neutral", "neutral"],
                          ["neutral", "neutral", "", "neutral"],
                          ["neutral", "neutral", "neutral", ""]])

trip_dataset = data_gen.generate_trip_dataset(n_baskets,
                                             assortments_matrix)

In [ ]:
M = np.zeros((n_items,n_items))
for trip in trip_dataset.trips:
    basket = trip.purchases
    for i in basket:
        for j in basket:
            if i!= j:
                M[i,j] += 1

M = M / np.sum(M, axis=1, keepdims=True)
M = np.nan_to_num(M, nan=0.0)
np.set_printoptions(precision=2, suppress=True)

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 5))
plt.imshow(M, cmap='magma', interpolation='nearest')
plt.colorbar(label='P(i|j)')
plt.title('Conditional Probability Heatmap P(i|j)')
plt.xlabel('j')
plt.ylabel('i')
plt.xticks(ticks=np.arange(8), labels=np.arange(1,9))
plt.yticks(ticks=np.arange(8), labels=np.arange(1,9))
plt.show()



In [ ]:
# Instantiate and train the model

model1 = AttentionBasedContextEmbedding(
    epochs=epochs,
    lr=lr,
    embedding_dim=embedding_dim,
    n_negative_samples=n_negative_samples
)
model1.instantiate(
    n_items=len(assortments_matrix[0]))
history = model1.fit(trip_dataset)


In [ ]:
# Visualize empirical distribution
import matplotlib.pyplot as plt

contexts = tf.constant([[i] for i in range(n_items)], dtype=tf.int32)
context_prediction = model1.predict(contexts)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(
    np.stack(context_prediction),
    vmin=0.0,
    vmax=np.max(np.stack(context_prediction)),
    cmap="Spectral",
)

axes[0].set_title("Model P(i|j) on elementary baskets")
plt.colorbar(im1, ax=axes[0])
axes[1].plot(history["train_loss"], label="Training Loss")
axes[1].set_xlabel("Training Steps")
axes[1].set_ylabel("Loss")
axes[1].set_title("Training Loss History")

plt.tight_layout()
plt.show()

In [ ]:
# Create evaluation dataset
eval_dataset = data_gen.generate_trip_dataset(100)

# Evaluate model
loss_eval_dataset_1 = model1.evaluate(eval_dataset)
print(f"Loss of model1 on the evaluation dataset {loss_eval_dataset_1}")

# Save model
model1.save_model("attn_model.json")

In [ ]:
# Create a second model without instantiating
model2 = AttentionBasedContextEmbedding(
    epochs=epochs,
    lr=lr,
    embedding_dim=embedding_dim,
    n_negative_samples=n_negative_samples
)

# Load first model and compare results on evaluation dataset
model2.load_model("attn_model.json")
loss_eval_dataset_2 = model2.evaluate(eval_dataset)
print(f"Loss of model2 on the evaluation dataset {loss_eval_dataset_2}")
os.remove("attn_model.json")